Health Care Blue Book Notebook to read in data from three years and clean output to get top HCPCS codes

In [1]:
import pandas as pd 
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns

pd.options.display.max_columns = 999

In [2]:
%matplotlib inline

Function to use Matt Barnes's code for lists and dictionaries

In [3]:
# Dictionary function to call during cleaning and setting up top lists
def getlist(dict):
    """Converts the keys from a dictionary into a list."""
    list = []
    for key in dict.keys():
        list.append(key)
    return list

Import data extracting from zip files

In [4]:
# import zipfile
# with zipfile.ZipFile("../data/Medicare_Provider_Util_Payment_PUF_CY2016.zip", "r") as zip_payments_2016:
#     zip_payments_2016.extractall("../data/")   

In [5]:
# import zipfile
# with zipfile.ZipFile("../data/Medicare_Provider_Util_Payment_PUF_CY2015.zip", "r") as zip_payments_2015:
#     zip_payments_2015.extractall("../data/")

In [6]:
# Import 2017 Payment file to work on data frame structure for one year
# To get top codes need all rows; nrows=100000 optional
prov_pmnt_2017 = pd.read_csv('../data/Medicare_Provider_Util_Payment_PUF_CY2017.txt', sep='\t',
header=0, dtype={'nppes_provider_zip': 'str','nppes_entity_code' : 'category', 'place_of_service' : 'category'}) 

prov_pmnt_2017 = prov_pmnt_2017.drop([0])
prov_pmnt_2017 = prov_pmnt_2017.reset_index(drop=True)
pd.set_option('display.max_columns', None)
prov_pmnt_2017.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_Medicare_allowed_amt,average_submitted_chrg_amt,average_Medicare_payment_amt,average_Medicare_standard_amt
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99217,Hospital observation care discharge,N,100.0,96.0,100.0,73.398800,325.780000,56.827200,57.492400
1,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99218,Hospital observation care typically 30 minutes,N,26.0,25.0,26.0,100.080000,449.000000,78.460000,79.306154
2,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99219,Hospital observation care typically 50 minutes,N,52.0,51.0,52.0,136.380000,614.000000,102.807692,103.895385
3,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99220,Hospital observation care typically 70 minutes...,N,59.0,59.0,59.0,190.363729,755.932203,141.293559,142.865763
4,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99221,"Initial hospital inpatient care, typically 30 ...",N,16.0,16.0,16.0,101.680000,462.812500,79.710000,80.750000


In [7]:
# Format column headers to lower case (place in the read in code)
# prov_pmnt_2017.columns = prov_pmnt_2017.columns.str.strip().str.lower()

Chunks code to use for sampling and when bring in Hospital data (optional)

In [8]:
# CHUNKS to review smaller sets of rows (change in the chunksize when read)
# look at the column types - first five rows in chunks

# df= pd.read_excel('data/MUP_OHP_R19_P04_V10_D17_APC_Provider.xlsx', skiprows = [0, 1, 2, 3, 4])
    
# op_2017_chunks = pd.read_excel(r'C:\Users\annru\Documents\healthcare-bluebook-the-unquantifiables\data\MUP_OHP_R19_P04_V10_D17_APC_Provider.xlsx', skiprows = [0, 1, 2, 3, 4], chunksize=5)
# op_2017_chunks1 = next(op_2017_chunks)  # payments1 is a dataframe of the first 5 rows

# for i, t in enumerate(payments1.iloc[0]):
#     print(type(t))

In [9]:
# need to check which columns have mixed data (10)
prov_pmnt_2017.dtypes

npi                                    int64
nppes_provider_last_org_name          object
nppes_provider_first_name             object
nppes_provider_mi                     object
nppes_credentials                     object
nppes_provider_gender                 object
nppes_entity_code                   category
nppes_provider_street1                object
nppes_provider_street2                object
nppes_provider_city                   object
nppes_provider_zip                    object
nppes_provider_state                  object
nppes_provider_country                object
provider_type                         object
medicare_participation_indicator      object
place_of_service                    category
hcpcs_code                            object
hcpcs_description                     object
hcpcs_drug_indicator                  object
line_srvc_cnt                        float64
bene_unique_cnt                      float64
bene_day_srvc_cnt                    float64
average_me

In [10]:
prov_pmnt_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9847443 entries, 0 to 9847442
Data columns (total 26 columns):
npi                                 int64
nppes_provider_last_org_name        object
nppes_provider_first_name           object
nppes_provider_mi                   object
nppes_credentials                   object
nppes_provider_gender               object
nppes_entity_code                   category
nppes_provider_street1              object
nppes_provider_street2              object
nppes_provider_city                 object
nppes_provider_zip                  object
nppes_provider_state                object
nppes_provider_country              object
provider_type                       object
medicare_participation_indicator    object
place_of_service                    category
hcpcs_code                          object
hcpcs_description                   object
hcpcs_drug_indicator                object
line_srvc_cnt                       float64
bene_unique_cnt        

In [11]:
# CHUNKS to review smaller sets of rows (change in the chunksize when read)
# look at the column types - first five rows in chunks
    
payments_chunks = pd.read_csv(r'C:\Users\annru\Documents\healthcare-bluebook-the-unquantifiables\data\Medicare_Provider_Util_Payment_PUF_CY2017.txt', sep='\t', header=0, chunksize=5)
payments1 = next(payments_chunks)  # payments1 is a dataframe of the first 5 rows

for i, t in enumerate(payments1.iloc[0]):
    print(type(t))

<class 'numpy.int64'>
<class 'str'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'numpy.float64'>
<class 'float'>
<class 'float'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>


In [12]:
# Payment data for 2016
prov_pmnt_2016 = pd.read_csv('../data/Medicare_Provider_Util_Payment_PUF_CY2016.txt', sep='\t', header=0,
nrows=100000, dtype={'nppes_provider_zip': 'str','nppes_entity_code' : 'category', 'place_of_service' : 'category'}  
                            )
prov_pmnt_2016 = prov_pmnt_2016.drop([0])
prov_pmnt_2016 = prov_pmnt_2016.reset_index(drop=True)
pd.set_option('display.max_columns', None)

prov_pmnt_2016.columns = prov_pmnt_2016.columns.str.strip().str.lower()
prov_pmnt_2016.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,average_medicare_standard_amt
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854.0,MD,US,Internal Medicine,Y,F,99217,Hospital observation care discharge,N,57.0,55.0,57.0,72.743158,325.157895,54.474386,55.807193
1,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854.0,MD,US,Internal Medicine,Y,F,99219,Hospital observation care typically 50 minutes,N,38.0,38.0,38.0,135.010000,614.000000,105.850000,107.230000
2,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854.0,MD,US,Internal Medicine,Y,F,99220,Hospital observation care typically 70 minutes...,N,23.0,23.0,23.0,189.239565,769.347826,146.116522,144.278696
3,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854.0,MD,US,Internal Medicine,Y,F,99221,"Initial hospital inpatient care, typically 30 ...",N,20.0,20.0,20.0,100.750000,460.000000,78.990000,80.280000
4,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854.0,MD,US,Internal Medicine,Y,F,99222,"Initial hospital inpatient care, typically 50 ...",N,96.0,87.0,96.0,136.250000,625.000000,105.477604,107.007708


In [13]:
# Payment data for 2015
prov_pmnt_2015 = pd.read_csv('../data/Medicare_Provider_Util_Payment_PUF_CY2015.txt', sep='\t', header=0,
nrows=100000, dtype={'nppes_provider_zip': 'str','nppes_entity_code' : 'category', 'place_of_service' : 'category'} 
                            )
prov_pmnt_2015 = prov_pmnt_2015.drop([0])
prov_pmnt_2015 = prov_pmnt_2015.reset_index(drop=True)
pd.set_option('display.max_columns', None)

prov_pmnt_2015.columns = prov_pmnt_2015.columns.str.strip().str.lower()
prov_pmnt_2015.head()

,npi,nppes_provider_last_org_name,nppes_provider_first_name,nppes_provider_mi,nppes_credentials,nppes_provider_gender,nppes_entity_code,nppes_provider_street1,nppes_provider_street2,nppes_provider_city,nppes_provider_zip,nppes_provider_state,nppes_provider_country,provider_type,medicare_participation_indicator,place_of_service,hcpcs_code,hcpcs_description,hcpcs_drug_indicator,line_srvc_cnt,bene_unique_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_submitted_chrg_amt,average_medicare_payment_amt,average_medicare_standard_amt
0,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99217,Hospital observation care discharge,N,23.0,23.0,23.0,72.680000,328.000000,54.502609,55.280870
1,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99219,Hospital observation care typically 50 minutes,N,18.0,18.0,18.0,135.850000,614.000000,100.958889,102.348889
2,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99221,"Initial hospital inpatient care, typically 30 ...",N,59.0,58.0,59.0,101.365085,333.288136,79.469661,80.859831
3,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99222,"Initial hospital inpatient care, typically 50 ...",N,132.0,130.0,132.0,139.010455,356.492424,107.933409,107.969621
4,1003000126,ENKESHAFI,ARDALAN,NaN,M.D.,M,I,900 SETON DR,NaN,CUMBERLAND,215021854,MD,US,Internal Medicine,Y,F,99223,"Initial hospital inpatient care, typically 70 ...",N,220.0,215.0,220.0,205.185955,631.186364,157.908955,160.930409


In [14]:
prov_pmnt_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 26 columns):
npi                                 99999 non-null int64
nppes_provider_last_org_name        99999 non-null object
nppes_provider_first_name           95713 non-null object
nppes_provider_mi                   72998 non-null object
nppes_credentials                   93500 non-null object
nppes_provider_gender               95713 non-null object
nppes_entity_code                   99999 non-null category
nppes_provider_street1              99999 non-null object
nppes_provider_street2              45015 non-null object
nppes_provider_city                 99999 non-null object
nppes_provider_zip                  99999 non-null object
nppes_provider_state                99999 non-null object
nppes_provider_country              99999 non-null object
provider_type                       99999 non-null object
medicare_participation_indicator    99999 non-null object
place_of_service    

Look at unique Provider types

In [15]:
print(prov_pmnt_2017['provider_type'].unique())

['Internal Medicine' 'Pathology' 'Anesthesiology' 'Family Practice'
 'Obstetrics & Gynecology' 'General Surgery' 'Urology' 'Cardiac Surgery'
 'Physician Assistant' 'Physical Therapist in Private Practice'
 'Dermatology' 'Licensed Clinical Social Worker' 'Optometry'
 'Mass Immunizer Roster Biller' 'Physical Medicine and Rehabilitation'
 'Radiation Oncology' 'Infectious Disease' 'Orthopedic Surgery'
 'Certified Clinical Nurse Specialist' 'Endocrinology' 'Chiropractic'
 'Psychiatry' 'Diagnostic Radiology' 'Centralized Flu' 'Rheumatology'
 'General Practice' 'Neurology' 'Emergency Medicine' 'Nephrology'
 'Ophthalmology' 'Nurse Practitioner' 'Pulmonary Disease' 'Otolaryngology'
 'Plastic and Reconstructive Surgery' 'Psychologist, Clinical'
 'Cardiology' 'Ambulance Service Provider' 'Interventional Cardiology'
 'Gastroenterology' 'Hospitalist' 'Audiologist'
 'Certified Registered Nurse Anesthetist (CRNA)' 'Geriatric Medicine'
 'Neurosurgery' 'Interventional Radiology' 'Critical Care (Intensi

Look at Unique place_of_service and entity_code

In [16]:
print(prov_pmnt_2017['place_of_service'].unique())


[F, O]
Categories (2, object): [F, O]


In [17]:
print(prov_pmnt_2017['nppes_entity_code'].unique())

[I, O]
Categories (2, object): [I, O]


Study Zip Codes before turn into 5 digits if combine

In [18]:
# df["Column Name"].str.len().min()
prov_pmnt_2017['nppes_provider_zip'].str.len().min()

3.0

In [19]:
# df["Column Name"].str.len().max()
prov_pmnt_2017['nppes_provider_zip'].str.len().max()

12.0

Reduce to the 12 columns (before calculated columns)

In [20]:
#creating the df for the selected columns
prov_pmnt_2017 = prov_pmnt_2017[['npi', 'nppes_provider_last_org_name', 'nppes_entity_code', 'provider_type', 'place_of_service', 'hcpcs_code',
    'hcpcs_description', 'line_srvc_cnt', 'bene_day_srvc_cnt', 'average_medicare_allowed_amt', 'average_medicare_payment_amt']]
prov_pmnt_2017.head()

,npi,nppes_provider_last_org_name,nppes_entity_code,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_medicare_payment_amt
0,1003000126,ENKESHAFI,I,Internal Medicine,F,99217,Hospital observation care discharge,100.0,100.0,73.398800,56.827200
1,1003000126,ENKESHAFI,I,Internal Medicine,F,99218,Hospital observation care typically 30 minutes,26.0,26.0,100.080000,78.460000
2,1003000126,ENKESHAFI,I,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,52.0,52.0,136.380000,102.807692
3,1003000126,ENKESHAFI,I,Internal Medicine,F,99220,Hospital observation care typically 70 minutes...,59.0,59.0,190.363729,141.293559
4,1003000126,ENKESHAFI,I,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",16.0,16.0,101.680000,79.710000


Get Top HCPCS codes based on 'bene_day_srvc_cnt'

In [21]:
# top n rows ordered by multiple columns
# df.nlargest(3,['col_name','col_name'])
prov_pmnt_2017.nlargest(10,['bene_day_srvc_cnt', 'average_medicare_payment_amt'])

,npi,nppes_provider_last_org_name,nppes_entity_code,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_medicare_payment_amt
5247034,1538144910,LABORATORY CORPORATION OF AMERICA HOLDINGS,O,Clinical Laboratory,O,36415,Insertion of needle into vein for collection o...,1504224.0,1504215.0,2.999863,2.938301
5247038,1538144910,LABORATORY CORPORATION OF AMERICA HOLDINGS,O,Clinical Laboratory,O,80053,"Blood test, comprehensive group of blood chemi...",1349822.0,1349822.0,9.356791,9.163964
635499,1063497451,LABORATORY CORPORATION OF AMERICA HOLDINGS,O,Clinical Laboratory,O,36415,Insertion of needle into vein for collection o...,1199303.0,1199227.0,2.999882,2.937164
5247347,1538144910,LABORATORY CORPORATION OF AMERICA HOLDINGS,O,Clinical Laboratory,O,85025,"Complete blood cell count (red cells, white bl...",970963.0,970912.0,10.648078,10.431655
635503,1063497451,LABORATORY CORPORATION OF AMERICA HOLDINGS,O,Clinical Laboratory,O,80053,"Blood test, comprehensive group of blood chemi...",968459.0,968454.0,9.471614,9.272854
5247039,1538144910,LABORATORY CORPORATION OF AMERICA HOLDINGS,O,Clinical Laboratory,O,80061,"Blood test, lipids (cholesterol and triglyceri...",964597.0,964596.0,14.379430,14.078581
8793595,1891731626,QUEST DIAGNOSTICS CLINICAL LABORATORIES INC,O,Clinical Laboratory,O,36415,Insertion of needle into vein for collection o...,893702.0,893702.0,3.000000,2.939279
8793598,1891731626,QUEST DIAGNOSTICS CLINICAL LABORATORIES INC,O,Clinical Laboratory,O,80053,"Blood test, comprehensive group of blood chemi...",775154.0,775153.0,9.537128,9.335161
5247316,1538144910,LABORATORY CORPORATION OF AMERICA HOLDINGS,O,Clinical Laboratory,O,84443,"Blood test, thyroid stimulating hormone (TSH)",717945.0,717941.0,22.386608,21.926596
5225838,1538105366,"SONORA QUEST LABORATORIES, LLC.",O,Clinical Laboratory,O,36415,Insertion of needle into vein for collection o...,696237.0,696236.0,2.999997,2.938906


Get Top HCPCS codes based on 'average_medicare_payment_amt'

In [23]:
# top n rows ordered by multiple columns
# df.nlargest(3,['col_name','col_name'])
prov_pmnt_2017.nlargest(10,['average_medicare_payment_amt'])

,npi,nppes_provider_last_org_name,nppes_entity_code,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_medicare_payment_amt
4096258,1417915653,ACCREDO HEALTH GROUP INC,O,All Other Suppliers,O,J7205,"Injection, factor viii fc fusion protein (reco...",71.0,50.0,56684.351549,44439.098451
2992901,1306804547,ACCREDO HEALTH GROUP INC,O,All Other Suppliers,O,J7195,"Injection, factor ix (antihemophilic factor, r...",56.0,47.0,50004.963571,38021.988750
9068636,1922019561,CASCADE HEMOPHILIA CONSORTIUM,O,All Other Suppliers,O,J7192,"Factor viii (antihemophilic factor, recombinan...",90.0,74.0,45801.470000,35901.975556
4096257,1417915653,ACCREDO HEALTH GROUP INC,O,All Other Suppliers,O,J7195,"Injection, factor ix (antihemophilic factor, r...",43.0,35.0,43775.277442,34319.817442
5365306,1548282510,ACCREDO HEALTH GROUP INC,O,All Other Suppliers,O,J7192,"Factor viii (antihemophilic factor, recombinan...",65.0,58.0,42626.505846,33410.026308
2992899,1306804547,ACCREDO HEALTH GROUP INC,O,All Other Suppliers,O,J7187,"Injection, von willebrand factor complex (huma...",67.0,49.0,42280.125522,33138.629701
8701143,1881662856,"SILICON VALLEY SURGERY CENTER, LP",O,Ambulatory Surgical Center,F,69930,Implantation of cochlear device,13.0,13.0,41333.160000,32405.200000
4096255,1417915653,ACCREDO HEALTH GROUP INC,O,All Other Suppliers,O,J7187,"Injection, von willebrand factor complex (huma...",40.0,31.0,40932.956000,32091.437500
2872867,1295168276,MANKIN,I,Physician Assistant,O,Q2043,"Sipuleucel-t, minimum of 50 million autologous...",27.0,27.0,39864.172593,31253.514074
7381354,1740596105,FORSYTHE,I,Radiation Oncology,O,Q2043,"Sipuleucel-t, minimum of 50 million autologous...",38.0,38.0,39752.548947,31158.451579


Add three measure columns - mean of the amt columns: 
'medicare_allowed_avg' (average_medicare_allowed_amt') ,
'medicare_srvc_day_avg' ('bene_day_srvc_cnt') ,
'medicare_payment_avg' ('average_medicare_payment_amt')

In [24]:
#caclulating the avg (mean) of the measure columns 
#list_name = df.loc[: , "first_measure_column":"second_measure_column"]
#df['new_column_name'] = list_name.mean(axis=0); (axis 0 = calculates column wise and axis=1 calculates row wise)

prov_pmnt_2017['medicare_allowed_avg'] = prov_pmnt_2017['average_medicare_allowed_amt'].mean(axis=0)
prov_pmnt_2017['medicare_srvc_day_avg'] = prov_pmnt_2017['bene_day_srvc_cnt'].mean(axis=0)
prov_pmnt_2017['medicare_payment_avg'] = prov_pmnt_2017['average_medicare_payment_amt'].mean(axis=0)

prov_pmnt_2017.head()

,npi,nppes_provider_last_org_name,nppes_entity_code,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_medicare_payment_amt,medicare_allowed_avg,medicare_srvc_day_avg,medicare_payment_avg
0,1003000126,ENKESHAFI,I,Internal Medicine,F,99217,Hospital observation care discharge,100.0,100.0,73.398800,56.827200,101.599329,140.334877,77.534799
1,1003000126,ENKESHAFI,I,Internal Medicine,F,99218,Hospital observation care typically 30 minutes,26.0,26.0,100.080000,78.460000,101.599329,140.334877,77.534799
2,1003000126,ENKESHAFI,I,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,52.0,52.0,136.380000,102.807692,101.599329,140.334877,77.534799
3,1003000126,ENKESHAFI,I,Internal Medicine,F,99220,Hospital observation care typically 70 minutes...,59.0,59.0,190.363729,141.293559,101.599329,140.334877,77.534799
4,1003000126,ENKESHAFI,I,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",16.0,16.0,101.680000,79.710000,101.599329,140.334877,77.534799


In [26]:
#rounding the measure columns to 2 decimal places
cols = ['average_medicare_allowed_amt', 'average_medicare_payment_amt','medicare_allowed_avg','medicare_payment_avg',
        'bene_day_srvc_cnt', 'medicare_srvc_day_avg']
prov_pmnt_2017[cols] = prov_pmnt_2017[cols].round(2)
prov_pmnt_2017.head()

,npi,nppes_provider_last_org_name,nppes_entity_code,provider_type,place_of_service,hcpcs_code,hcpcs_description,line_srvc_cnt,bene_day_srvc_cnt,average_medicare_allowed_amt,average_medicare_payment_amt,medicare_allowed_avg,medicare_srvc_day_avg,medicare_payment_avg
0,1003000126,ENKESHAFI,I,Internal Medicine,F,99217,Hospital observation care discharge,100.0,100.0,73.40,56.83,101.6,140.33,77.53
1,1003000126,ENKESHAFI,I,Internal Medicine,F,99218,Hospital observation care typically 30 minutes,26.0,26.0,100.08,78.46,101.6,140.33,77.53
2,1003000126,ENKESHAFI,I,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,52.0,52.0,136.38,102.81,101.6,140.33,77.53
3,1003000126,ENKESHAFI,I,Internal Medicine,F,99220,Hospital observation care typically 70 minutes...,59.0,59.0,190.36,141.29,101.6,140.33,77.53
4,1003000126,ENKESHAFI,I,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",16.0,16.0,101.68,79.71,101.6,140.33,77.53
